# Coursework 2 
### Perform k-nearest neighbour (KNN) retrieval experiments according to standard practices in pattern recognition. Use retrieval error (ie @rank1, @rank10) as the performance metric to evaluate different methods. Your baseline approach is KNN on provided features. Use distance metric learning methods to improve a baseline performance.

#### So, firstly, we need to create a baseline, which is KNN on provided features. 

In [1]:
from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()

## Load features

In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt

with open("PR_data/feature_data.json", "r") as file:
    features = json.load(file)
    
data_features = np.asarray(features)

print('Data shape: {}'.format(data_features.shape))

Data shape: (14096, 2048)


## Load data

In [3]:
#Load labels
labelss = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()

#Load camId
cam_Ids = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

#Load indexes
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()

#Load training indexes
print("training indexes : {}".format(train_idxs.shape))
print("query indexes : {}".format( query_idxs.shape))
print("gallery indexes : {}".format( gallery_idxs.shape))

training indexes : (7368,)
query indexes : (1400,)
gallery indexes : (5328,)


In [4]:
#converting to python index notation

train_idxs = train_idxs - 1
query_idxs = query_idxs - 1
gallery_idxs = gallery_idxs - 1

## Separate the features and data into training, query and gallery sets 

In [7]:
train_f = []
train_label = []
train_camid = []
# divide the features data into training 
for i in range (len(train_idxs)):
    train_f.append(data_features[train_idxs[i]])
    train_camid.append(cam_Ids[train_idxs[i]])
    train_label.append(labelss[train_idxs[i]])
    
train_f = np.asarray(train_f)    
train_label = np.asarray(train_label)
train_camid = np.asarray(train_camid)

In [8]:
#features, cam, label data into gallery
query_f = []
query_camid = []
query_label = []

for i in range (len(query_idxs)):
    query_f.append(data_features[query_idxs[i]])
    query_camid.append(cam_Ids[query_idxs[i]])
    query_label.append(labelss[query_idxs[i]])
    
query_f = np.asarray(query_f) 
query_label = np.asarray(query_label)
query_camid = np.asarray(query_camid)

In [9]:
#features, cam, label data into gallery
gallery_f = []
gallery_camid = []
gallery_label = []

for i in range (len(gallery_idxs)):
    gallery_f.append(data_features[gallery_idxs[i]])
    gallery_camid.append(cam_Ids[gallery_idxs[i]])
    gallery_label.append(labelss[gallery_idxs[i]])
       
gallery_f = np.asarray(gallery_f)  
gallery_camid = np.asarray(gallery_camid)  
gallery_label = np.asarray(gallery_label)  

### Concatenate query and gallery features/labels/cam ids

In [10]:
#stacking labels, cam ids and features

query = np.vstack((query_f.T, query_label, query_camid)) #transpose feature matrix to match dimensions 
gallery = np.vstack((gallery_f.T, gallery_label, gallery_camid)) #transpose feature matrix to match dimensions 

#transpose back to get right shape
query = query.T
gallery = gallery.T

print(query[:,-2])
print(gallery.shape)

[   3.    3.    6. ... 1461. 1463. 1463.]
(5328, 2050)


## Remove the repetitions of cam ids and labels and perform NN classification

## Nearest Neighbor Classification

In [105]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

k = 10 #number of nearest neighbors parameter

rank = []
distance = []
idx = []

import time
start_time = time.time()

##sqeuclidean instead of euclidean for computing efficientcy. As we are only interested in the ranklists
##and not the actual values of the distances.

for i in range (len(query[:,0])):
    
    ##getting rid of label/camid repetions 
    gallery_no_rep = gallery[~np.logical_and((gallery[:,-1] == query[i, -1]), (gallery[:,-2] == query[i, -2]))]
    
    NN = NearestNeighbors(n_neighbors = k, metric = 'sqeuclidean') #setting up NN
    NN.fit(gallery_no_rep[:,:-2], gallery_no_rep[:,-2])

   
    query_f_test = query_f[i,:].reshape(1,-1)
    dist, idxs = NN.kneighbors(query_f_test)
    
    distance.append(dist)
    idx.append(idxs)

distance = np.asarray(distance)
idx = np.asarray(idx)

print(idx.shape)

##creating ranklist of predicted labels (1400x10)
for i in range (len(query[:,0])):
    row_rank = []
    for j in range (len(idx[0,:])):
        row_rank.append(gallery[idx[i,j],-2])
    rank.append(row_rank)
    
rank = np.asarray(rank)     

    
print("--- %s seconds ---" % (time.time() - start_time))

print("shape of rank matx: %s" % (rank.shape))

(1400, 1, 10)
--- 158.0308358669281 seconds ---
[[   3]
 [   3]
 [   6]
 ...
 [1461]
 [1463]
 [1463]]
(1400, 1, 10)


In [106]:
rank = np.squeeze(rank)
print(rank.shape)
print(rank)

(1400, 10)
[[1367.  739. 1426. ... 1449.   51. 1367.]
 [ 509.  232.  232. ... 1426.  171.    3.]
 [   6.    6.    6. ...  678.  675.  647.]
 ...
 [1418. 1418. 1461. ... 1461. 1418. 1293.]
 [1463. 1463. 1463. ...  884.  884.  905.]
 [1463. 1463.  884. ...  884.  346.  905.]]


## Creating a Match/No Match array 

In [143]:
query_label = query_label.reshape(1400,1)

rank_bin1 = []

for i in range (len(rank[:,0])):
    rank_bin = []
    for j in range (len(rank[0,:])): 
        if (rank[i,j] == query_label[i]):   ##if match, set 1, otherwise set 0
            rank_bin.append(1)
        else:
            rank_bin.append(0)
    rank_bin1.append(rank_bin)
    
    
rank_bin1 = np.asarray(rank_bin1)   ##array of label matches and mismatches

## Rank calculation

In [149]:
rank_1 = np.sum(rank_bin1[:,0])/1400
r5 = rank_bin1[:,:5].sum(axis = 1)/5
rank_5 = np.count_nonzero(r5)/1400
r10 = rank_bin1[:,:10].sum(axis = 1)/5
rank_10 = np.count_nonzero(r10)/1400

print("Accuracy")
print("rank 1: %s" % (rank_1))
print("rank 5: %s" % (rank_5))
print("rank 10: %s" % (rank_10))

Accuracy
rank 1: 0.47
rank 5: 0.6735714285714286
rank 10: 0.7564285714285715


### Finding precisions and recalls for calculating the mAP